필요한 모듈 임포트

In [1]:
import os
import shutil
from tqdm import tqdm

데이터 탐색

In [32]:
DATA_DIR = '.\data'

In [ ]:
if os.path.isdir(".\speech_commands_v0.01.tar"):
    if not os.path.isdir(DATA_DIR):
        os.rename(".\speech_commands_v0.01.tar", DATA_DIR)

In [ ]:
os.listdir(DATA_DIR)

In [ ]:
for name in os.listdir(DATA_DIR):
    if name.endswith("_list.txt"):
        print(name)

In [34]:
def get_list(DATA_DIR: str, list_name: str):
    with open(os.path.join(DATA_DIR, list_name)) as f:
        return f.readlines()

In [35]:
test_list = get_list(DATA_DIR, 'testing_list.txt')
val_list = get_list(DATA_DIR, 'validation_list.txt')

In [36]:
len(test_list), len(val_list)

(6835, 6798)

In [ ]:
test_list[0]

In [ ]:
os.path.dirname(test_list[0])

In [ ]:
os.path.basename(test_list[0])[:-1]

In [ ]:
# make_dir(os.getcwd(), 'data_COPY')

In [ ]:
# shutil.copytree('.\data', '.\data_COPY')

Data Split

In [53]:
class DataLoader:
    def __init__(self,
                BASE: str,
                NEW_BASE: str):
        self.BASE = BASE
        self.NEW_BASE = NEW_BASE

    # speech dataset에 포함되어 있는 testing_list.txt와 validation_list.txt 텍스트 파일을 리스트로 가져오기    
    def get_list(self):
        ls_for_ls = []
        cnt = -1
        for file_name in os.listdir(self.BASE):
            if file_name.endswith("_list.txt"):
                with open(os.path.join(self.BASE, file_name)) as f:
                    data_list = f.readlines()
                ls_for_ls.append(data_list)
                cnt += 1
                print('{} 데이터 리스트가 {}번 인덱스에 저장되었습니다.'.format(file_name.split('_')[0], cnt))
        return ls_for_ls
    
    # 원하는 디렉토리가 있는 지 확인 후, 없으면 만들어주는 함수. 만들어진 디렉토리의 path를 str 형태로 받아볼 수 있다.
    def make_dir(self, path, folder_name):
        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            os.mkdir(folder_path)
        return folder_path

    # 리스트 안에 있는 파일을 지정한 경로로 옮기는 함수
    def fileinlist(self, 
                   file_list: list, dataset_dir: str):
        print('{} 폴더를 구성하는 중입니다.'.format(os.path.basename(dataset_dir)))
        for f in file_list:
            file_dir = os.path.dirname(f)
            file_name = os.path.basename(f)[:-1] # 파일명 끝에 '\n'이 붙어있어 제거한다.
            self.make_dir(dataset_dir, file_dir)
            
            old_path = os.path.join(self.BASE, f[:-1])
            new_path = os.path.join(dataset_dir, f[:-1])
            shutil.move(old_path, new_path)
        print('{} 폴더 구성이 완료 되었습니다.'.format(os.path.basename(dataset_dir)))
    
    def split(self):
        test_list, val_list = self.get_list()
        
        os.mkdir(self.NEW_BASE)
        test_path = self.make_dir(self.NEW_BASE, 'test')
        train_path = self.make_dir(self.NEW_BASE, 'train')
        val_path = self.make_dir(self.NEW_BASE, 'val')
        self.fileinlist(test_list, test_path)
        self.fileinlist(val_list, val_path)
               
    # 리스트 안의 test data와 validation의 data를 먼저 모두 옮기고 남은 데이터를 한꺼번에 지정한 경로의 train 폴더로 옮긴다.    

        print('{} 폴더를 구성하는 중입니다.'.format(os.path.basename(train_path)))  
        # move train-set
        for folder in tqdm(os.listdir(self.BASE)):
            old_path = os.path.join(self.BASE, folder)
            new_path = os.path.join(train_path, folder)
            if os.path.isdir(old_path):
                shutil.move(old_path, new_path)
        print('{} 폴더 구성이 완료 되었습니다.'.format(os.path.basename(train_path)))
        print('데이터 분할이 완료 되었습니다.')
        

In [54]:
BASE = '../../data/dafit/DF_903'
NEW_BASE = '../../data/dafit/DF_903_split'

In [55]:
data_loader = DataLoader(BASE, NEW_BASE)
data_loader.split()

testing 데이터 리스트가 0번 인덱스에 저장되었습니다.
validation 데이터 리스트가 1번 인덱스에 저장되었습니다.
test 폴더를 구성하는 중입니다.
test 폴더를 구성이 완료 되었습니다.
val 폴더를 구성하는 중입니다.



100%|█████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 369.41it/s]

val 폴더를 구성이 완료 되었습니다.
train 폴더를 구성하는 중입니다.
train 폴더를 구성이 완료 되었습니다.
데이터 분할이 완료 되었습니다.


TypeError: len_chk() takes 1 positional argument but 2 were given